In [2]:
#DATA PREPROCESSING FOR LINEAR REGRESSION
import numpy as np
import pandas as pd
df=pd.read_csv("salepricedata.csv")
#top_threshold = df['SalePrice'].quantile(0.99)

# Remove values above the top threshold
#df = df[df['SalePrice'] <= top_threshold]
#Lookout for categorical variables
#List to store variables of different types
columns_categorical_variables=[]
columns_nominal_variables=[]
columns_ordinal_variables=[]
columns_continuous_variables=[]
#Loop across all columns and consider any column with less than 200 unique value as a categorical feature.
#Print report combined with list updation of categorical variables
print("Report of all categorical variables")
for i in df.columns:
    #if count of unique values is less than 200 consider it to be categorical
    if len(df[i].unique())<200:
        #if the first element of the unique value is a number or a float, consider it an ordinal data
        if type(df[i].unique()[0])==np.int64 or type(df[i].unique()[0])==np.float64:
            print("variable type=ordinal")
            columns_ordinal_variables.append(i)
            #For columns having ordinal data fill na values as 0.
            df[i]=df[i].fillna(0)
            #df[i] = df[i].fillna(df[i].mode()[0])
        #else consider it nominal data
        else:
            print("variable type=nominal")
            columns_nominal_variables.append(i)
            #For columns having nominal data fill na values as "NULL".This will be considered as an additional category.
            df[i]=df[i].fillna("NULL")
            #df[i] = df[i].fillna(df[i].mode()[0])
        #Update categorical variables. Might be of use. Print categorical variables and see for any misclassifications.
        columns_categorical_variables.append(i)
        print("column=", i)
        print("unique values=", df[i].unique())
        print("count of unique values=", len(df[i].unique()))
        #For output readability. Ignore the next two prints.
        print(" ")
        print(" ")
#Print the nominal and ordinal variables
print("nominal variables")
print(columns_nominal_variables)
print("ordinal variables")
print(columns_ordinal_variables)

Report of all categorical variables
variable type=ordinal
column= MSSubClass
unique values= [ 20 160 120  60  30  50  90  80 190  45  85  70  75 180  40]
count of unique values= 15
 
 
variable type=nominal
column= MSZoning
unique values= ['RL' 'FV' 'RM' 'C (all)' 'RH']
count of unique values= 5
 
 
variable type=ordinal
column= LotFrontage
unique values= [ 60.  40.  80.  55.  62.   0. 101.  86.  70.  75.  53.  91. 128.  85.
  65.  72.  68. 110.  64.  63. 103.  76.  69.  92.  50.  52.  97.  57.
  90. 130. 150.  24.  77.  82.  51.  73. 106.  84.  78.  87.  61.  48.
  98.  99.  21.  58. 102. 100.  35.  44.  41.  88.  43. 129. 108.  89.
 107.  66. 174.  79.  36. 121.  81. 138.  74.  34.  95.  37.  54.  59.
  71.  56.  96. 168.  45. 124.  93.  67.  38. 120.  94.  30.  32.  83.
  39. 149. 313. 114.  47. 111. 105.  42. 122.  46. 116. 141. 137. 118.
 182. 160. 140. 152.  49. 109.]
count of unique values= 104
 
 
variable type=nominal
column= Street
unique values= ['Pave' 'Grvl']
count of uniq

In [3]:
# Look at continuous variables. If a feature has more than 200 unique values, it is continuous.
for i in df.columns:
    if len(df[i].unique()) > 200:
        # For continuous variables, replace null values with the mean of the column
        mean_value = df[i].mean()
        #df[i] = df[i].fillna(mean_value)
        df[i] = df[i].fillna(0)
        print("Column =", i)
        print("Unique values =", df[i].unique())
        print("Count of unique values =", len(df[i].unique()))
        print(" ")

        # Append all continuous variables except the target
        if i != "SalePrice":
            columns_continuous_variables.append(i)

Column = Id
Unique values = [  1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18
  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36
  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53  54
  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71  72
  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89  90
  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107 108
 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126
 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144
 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162
 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180
 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198
 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216
 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234
 235 236 237 238 239 24

In [4]:
#For nominal variables, create dummy features with 0 and 1 as values to avoid error in linear regression results
#This line of code creates a dummy variable for all nominal varables that is stored in a list.
df_d=pd.get_dummies(data=df, columns=columns_nominal_variables, dtype=np.int64)
#As new columns for dummy variables are created at the end of the dataframe, add the output column to the end of the df.
df_d["Output_Sale_Price"]=df_d["SalePrice"]
df_d=df_d.drop("SalePrice", axis='columns')
#Preview df_d. This is the final dataset before we proceed with modelling steps
df_d.head()

# Calculate the lower and upper bounds for outlier removal
#lower_bound = df_d["Output_Sale_Price"].quantile(0.05)
upper_bound = df_d["Output_Sale_Price"].quantile(0.95)
lower_bound = df_d["Output_Sale_Price"].quantile(0.01)
#upper_bound = df_d["Output_Sale_Price"].quantile(0.95)
# Keep only the values within the specified percentile range
df_d = df_d[(df_d["Output_Sale_Price"] >= lower_bound) & (df_d["Output_Sale_Price"] <= upper_bound)]

# Preview the modified df_d
df_d.head()


,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial,Output_Sale_Price
0,1,20,60.0,8430,5,5,1978,1978,136.0,616,...,0,0,1,0,0,0,0,1,0,124000
1,2,160,40.0,3951,6,5,2009,2009,76.0,0,...,1,0,0,0,0,0,0,0,1,164500
2,3,20,80.0,11600,6,5,1960,1960,175.0,565,...,0,0,1,0,0,0,0,1,0,145250
3,4,120,55.0,7892,6,5,1993,1993,0.0,300,...,0,0,1,0,0,0,0,1,0,153900
4,5,60,62.0,7415,6,5,2004,2004,0.0,759,...,0,0,1,0,0,0,0,1,0,193000


In [5]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
#set seed for replicable results. This way the same random numbers would be generated for test and train split.
import random
random.seed(11)
#Drop the id column as it is not meant to be a feature for prediction.
df_d=df_d.drop("Id", axis='columns')
#The last column is the output variable. Rest all are features
X = df_d.iloc[:, :-1]
y = df_d.iloc[:, -1]
#Split into training and test sets

###### AS WE DO NOT HAVE THE OUTPUT VARIABLE IN TEST SET, WE ASSUME THE TRAINING SET TO BE THE SUPERSET.
###### THE DATAFRAME IMPORTED HERE IS THE TRAIN.CSV UPLOADED IN BRIGHTSPACE RENAMED AS SALEPRICEDATA.CSV.
###### THE TEST SET FROM BRIGHTSPACE WOULD BE THE VALIDATION OR THE FINAL UNSEEN TEST SET.
###### WE WILL PERFORM MODEL EVALUATION/ CROSS VALIDATION ON OUR LOCAL TEST TEST.
###### THE BEST POSSIBLE RESULT (r^2) WE SEE ON OUR LOCAL TEST WOULD ALSO HAVE TO BE REPLICATED ON THE UNSEEN
###### TEST SET. HOPEFULLY.

#Split into training and test sets. test set would be 10% the size of the dataset. Off the top of my head.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=0)
#Fit simple regression model.
regr = LinearRegression()
regr.fit(X_train, y_train)
#Predict y on local test set
y_pred=regr.predict(X_test)
#Preview predicted sale prices
print("Predicted Prices")
print(y_pred)
print(" ")
print("Actual Prices")
print(y_test)

Predicted Prices
[253251.2448182  214706.31660322 153313.37653998 221997.24260684
 151270.05107433 183273.87803296 208173.17946024 120999.74542403
  68072.15296833 163352.57074631 167796.15863119 198454.70062942
 245254.93187279 139434.82051035 175554.68843434 137572.54297283
 127038.89936592  93872.009438   250555.64673081 159254.11720574
 113210.42967627 149877.59558548 168634.39196011 125800.15057101
 135816.21770416 129263.73744951 152269.3924653  188569.76483444
 210000.1668809  181716.69051957 131587.56259539 216836.62294482
 160518.78670303 191870.85762816 199827.43267326 116896.68229075
 126204.75215257  86570.09713793 141457.52673535 144535.0269469
 215454.88648188 236596.11543801 147312.30002061 119917.36890589
 156501.84356572 265211.88608023 150550.12267541 285489.82202171
 105289.07883542 256223.83507953 122617.75370059 209458.06409937
 175615.90265317 207814.34341194 199197.16841015 128542.84326536
 181872.04095166 194082.27059816 211880.30612998 136725.51804925
 169293.3

In [6]:
#Determine r2
from sklearn.metrics import r2_score
print(r2_score(y_test, y_pred))
#Other way to do the same thing
print(regr.score(X_test, y_test))

0.8682781730853287
0.8682781730853287


In [7]:
#We get a reasonable r2 of 0.8704 on the test set!
#Now, train a new model on the entire training data available and predict the actual unseen test set(Validation) values.
#Process the test set the same way it was done earlier for the original train dataset
#This model is decent enough so we ignore paramter tuning for now. The model can be improved futher, later.


In [8]:
#DATA PREPROCESSING FOR ACTUAL TEST/VALIDATION SET
import numpy as np
import pandas as pd
df_val=pd.read_csv("test_val.csv")
#The columns that are categorical do not change. Estimating it again might result in inconsistent results.
columns_categorical_variables_val=columns_categorical_variables
columns_nominal_variables_val=columns_nominal_variables
columns_ordinal_variables_val=columns_ordinal_variables
columns_continuous_variables_val=columns_continuous_variables
#For all categorical and continuous variables fillna based on if they are ordinal or nominal(0,NULL) pr continuous(0)
for i in columns_ordinal_variables_val:
        df_val[i]=df_val[i].fillna(0)
for i in columns_nominal_variables_val:
        df_val[i]=df_val[i].fillna("NULL")
for i in columns_continuous_variables_val:
        mean_value = df_val[i].mean()
        #df_val[i]=df_val[i].fillna(mean_value)
        df_val[i]=df_val[i].fillna(0)
#create dummy variables for nominal variables
df_d_val=pd.get_dummies(data=df_val, columns=columns_nominal_variables_val, dtype=np.int64)
#drop ID as it is not meant to exist. No.
df_d_val=df_d_val.drop("Id", axis="columns")
df_d_val
#Equalize the feature vectors for test and train set: If a category does not exist in any column a dummy column might be
#missing and the model collapses.
#train_col  test_col
# 1           1
# 2           2
# 3
#would result in
#train_col_1 train_col_2 train_col_3    test_col_1 test_col_1
#    1          0            0              1          0
#    0          1            0              0          1
#    0          0            1
#We don't want that. So remove features that are not seen in both of the columns.
for i in X.columns:
    if i not in df_d_val.columns:
        X=X.drop(i, axis="columns")
for i in df_d_val.columns:
    if i not in X.columns:
        df_d_val=df_d_val.drop(i, axis="columns")

In [9]:
#train a new model on the entire training data available and predict the actual unseen test set(Validation) values.
regr_final = LinearRegression()
regr_final.fit(X, y)
#predict the scores for the actual validation/test set
y_val_pred=regr_final.predict(df_d_val)
#Eureka! This cannot be verified by us. All hail Frank, the TA.
print(y_val_pred)

[ 59454.07884667 211333.38575988 327830.41215115 482440.11967429
  64830.3402621  176184.26711439 160036.52719732 234301.63045259
 176190.50472229 123855.45554825 134601.6577748  166125.89956593
 185661.37954127 157902.59242325 136938.03549338  85278.22748578
 196496.43803807 199545.86388324 167419.65715859 134175.40278153
 169212.54533969  60906.20174284 248612.32445343  68202.24766216
 187800.3390016  143902.63091463 333914.89907183 130868.38134375
 168953.09823972 334037.74861357 151043.47395525  68471.95125614
 154314.1293782  178711.06108349 307477.98096105 268298.47020542
 165800.78448661 205960.35160021 222769.37928038 176051.25331728
 178719.84805828 353719.84362319 102102.31901108 126527.82232188
 147934.2593883  110589.9535785  217117.38731696 140293.51726547
 136427.66731886 202405.67909026 152206.07669231 350819.62268618
 124599.95693176  99763.99284794 310025.71129213 244963.30659284
 171983.64345534 171055.27639687 115327.21344462 366777.22169087
 155383.50830004 127691.4

In [10]:
#Next task is to improve the model accuracy by using complex models(polynomial/lasso or ridge regression, CNN)
#Might have to make use of cross validation methods(GridsearchCV) and perform model tuning.

In [11]:
#DO NOT RUN THIS AND THE NEXT CELL IF TIME IS SPARSE. TAKES A LONG TIME TO RUN.
#MODEL ACCURACY IS FURTHER IMPROVED IN LASSO REGRESSION. NEURAL NETWORK IS NOT REALLY USEFUL HERE.
#from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import MinMaxScaler
#Scale the data as regularization methods in CNN are sensitive to scale/ranges of features.
scaler = MinMaxScaler()
X_train_scaled=scaler.fit_transform(X_train)
X_test_scaled=scaler.transform(X_test)
#hyperparameters tuned until convergence by random variation in test train split. Change only if accuracy improves.
#params = { 'hidden_layer_sizes' : [10,10],
#    'activation' : 'relu', 'solver' : 'adam',
#    'alpha' : 0.13, 'batch_size' : 10,
#    'random_state' : 0, 'tol' : 0.0001,
#    'nesterovs_momentum' : False,
#    'learning_rate' : 'constant',
#    'learning_rate_init' : 0.008,
#    'max_iter' : 2500, 'shuffle' : True,
#    'n_iter_no_change' : 50, 'verbose' : False }
#net = MLPRegressor(**params)
#net.fit(X_train_scaled, y_train)
#y_pred_nn=net.predict(X_test_scaled)
#print(r2_score(y_test, y_pred_nn))
#Accuracy improves by 0.5. But meh. Not worth it.

In [12]:
#fit the neural net model on the whole dataset and predict y on actual test/validation set
scaler_full_data = MinMaxScaler()
X_scaled=scaler_full_data.fit_transform(X)
df_d_val_scaled=scaler_full_data.transform(df_d_val)
#fit the same model
#params = { 'hidden_layer_sizes' : [10,10],
#    'activation' : 'relu', 'solver' : 'adam',
#    'alpha' : 0.13, 'batch_size' : 10,
#    'random_state' : 0, 'tol' : 0.0001,
#    'nesterovs_momentum' : False,
#    'learning_rate' : 'constant',
#    'learning_rate_init' : 0.008,
#    'max_iter' : 2500, 'shuffle' : True,
#    'n_iter_no_change' : 50, 'verbose' : False }
#net_final = MLPRegressor(**params)
#net_final.fit(X_scaled, y)
#y_val_pred_nn=net_final.predict(df_d_val_scaled)

In [13]:
#print(y_val_pred_nn)


In [14]:
#Next alternative is to try regression with regularization methods (Rigdge/Lasso)
#Fit lasso as lasso is better than ridge if the data has a lot of features. Ridge sets the penalty term to 0.
from sklearn.linear_model import Lasso
#Increaseing or decreasing the alpha by 1 would result in lesser r2. Model is converged at a test score of 91.24.
las = Lasso(alpha = 61)
las.fit(X_train_scaled, y_train)
#Predict y on local test set
y_pred_las=las.predict(X_test_scaled)
print(r2_score(y_test, y_pred_las))
#Lasso gives the best accuracy. A whopping 91.24! The accuracy on the actual test set should be 91.5 approximately
#Lasso just beat CNN. The mother of all algorithms! Hoping to probe more given time and inclination.
las_val = Lasso(alpha = 61)
las_val.fit(X_scaled, y)
y_val_pred_las=las_val.predict(df_d_val_scaled)
#print(y_val_pred_las)
df_submission1=df_val["Id"]
df_submission2=pd.DataFrame(data=y_val_pred_las, columns=["Predicted_Sale_Price_For_Test_Set"], index=None)
df_submission=pd.concat([df_submission1, df_submission2], axis=1)
print(df_submission)
#Uncomment the next lines to have a look at the difference in predictions in different models.
#print(" ")
#print(y_val_pred_nn)
#print(" ")
#print(y_val_pred)

# determining the name of the file for submission
file_name = 'ECON_Competition_Submission.xlsx'

# saving the excel
df_submission.to_excel(file_name)
#Find the submission file in the same directory!
df_submission

0.907092924710374
       Id  Predicted_Sale_Price_For_Test_Set
0     951                      109626.496089
1     952                      210617.265344
2     953                      323434.320018
3     954                      422353.478907
4     955                       77398.187937
..    ...                                ...
495  1446                      209801.946700
496  1447                      226828.426792
497  1448                      127386.863037
498  1449                       54903.063670
499  1450                      127936.597368

[500 rows x 2 columns]


,Id,Predicted_Sale_Price_For_Test_Set
0,951,109626.496089
1,952,210617.265344
2,953,323434.320018
3,954,422353.478907
4,955,77398.187937
...,...,...
495,1446,209801.946700
496,1447,226828.426792
497,1448,127386.863037
498,1449,54903.063670


In [15]:
df_submission

,Id,Predicted_Sale_Price_For_Test_Set
0,951,109626.496089
1,952,210617.265344
2,953,323434.320018
3,954,422353.478907
4,955,77398.187937
...,...,...
495,1446,209801.946700
496,1447,226828.426792
497,1448,127386.863037
498,1449,54903.063670


In [16]:
from sklearn.model_selection import GridSearchCV

# Set the alphas (alpha values to test during grid search)
alphas = np.arange(1, 100, 5)

# Create a Lasso model
lasso_model = Lasso()

# Set up the parameter grid
param_grid = {'alpha': alphas}

# Create a GridSearchCV object
grid_search = GridSearchCV(lasso_model, param_grid, cv=5, scoring='r2')
grid_search.fit(X_scaled, y)

# Get the best parameters and corresponding mean cross-validated score
best_alpha_grid = grid_search.best_params_['alpha']
best_score = grid_search.best_score_
print("Best alpha from GridSearchCV:", best_alpha_grid)
print("Best mean cross-validated score from GridSearchCV:", best_score)

# Fit a separate Lasso model on X_train_scaled with the best alpha from GridSearchCV
lasso_model_grid = Lasso(alpha=best_alpha_grid)
lasso_model_grid.fit(X_train_scaled, y_train)

# Evaluate the model on the local test set (X_test_scaled)
y_test_pred_lasso_grid = lasso_model_grid.predict(X_test_scaled)
r2_lasso_local_test_grid = r2_score(y_test, y_test_pred_lasso_grid)
print("R-squared on local test set using Lasso with best alpha from GridSearchCV:", r2_lasso_local_test_grid)

# Fit the Lasso model on the entire training dataset with the best alpha from GridSearchCV
lasso_final_grid = Lasso(alpha=best_alpha_grid)
lasso_final_grid.fit(X_scaled, y)

# Predict the final prices on df_d_val_scaled
y_val_pred_lasso_final_grid = lasso_final_grid.predict(df_d_val_scaled)

# Create a DataFrame for submission
df_submission_lasso_grid = pd.DataFrame({'Id': df_val['Id'], 'Predicted_Sale_Price_For_Test_Set': y_val_pred_lasso_final_grid})

# Save the submission to an Excel file
file_name_lasso_grid = 'ECON_Competition_Submission_Lasso_GridSearchCV.xlsx'
df_submission_lasso_grid.to_excel(file_name_lasso_grid, index=False)

# Display the submission DataFrame
df_submission_lasso_grid

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.761e+10, tolerance: 2.152e+08
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.116e+10, tolerance: 2.293e+08
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.088e+10, tolerance: 2.165e

Best alpha from GridSearchCV: 96
Best mean cross-validated score from GridSearchCV: 0.8448733345442087
R-squared on local test set using Lasso with best alpha from GridSearchCV: 0.8990082354581437


,Id,Predicted_Sale_Price_For_Test_Set
0,951,109987.666188
1,952,207709.523451
2,953,321686.042528
3,954,419032.495300
4,955,79477.648123
...,...,...
495,1446,209222.657420
496,1447,228845.958259
497,1448,127715.417264
498,1449,57867.692205


In [17]:
import xgboost as xgb
from sklearn.model_selection import GridSearchCV

param_grid_xgb = {
    'n_estimators': [200, 160],
    'learning_rate': [0.1],
    'max_depth': [4],
    'subsample': [0.7],
    'gamma': [0],
    'reg_alpha': [180],
    'reg_lambda': [140],
}

# Rest of the code remains the same


# Create an XGBoost regressor
xgb_regressor = xgb.XGBRegressor()

# Create a GridSearchCV object
grid_search_xgb = GridSearchCV(xgb_regressor, param_grid_xgb, cv=5, scoring='r2', verbose=1, n_jobs=-1)
grid_search_xgb.fit(X_scaled, y)

# Get the best parameters and corresponding mean cross-validated score
best_params_xgb = grid_search_xgb.best_params_
best_score_xgb = grid_search_xgb.best_score_
print("Best parameters from GridSearchCV for XGBoost:", best_params_xgb)
print("Best mean cross-validated score from GridSearchCV for XGBoost:", best_score_xgb)

# Fit a separate XGBoost model on X_train_scaled with the best parameters from GridSearchCV
xgb_model = xgb.XGBRegressor(**best_params_xgb)
xgb_model.fit(X_train_scaled, y_train)

# Evaluate the model on the local test set (X_test_scaled)
y_test_pred_xgb_grid = xgb_model.predict(X_test_scaled)
r2_xgb_local_test_grid = r2_score(y_test, y_test_pred_xgb_grid)
print("R-squared on local test set using XGBoost with best parameters from GridSearchCV:", r2_xgb_local_test_grid)

# Fit the XGBoost model on the entire training dataset with the best parameters from GridSearchCV
xgb_final = xgb.XGBRegressor(**best_params_xgb)
xgb_final.fit(X_scaled, y)

# Predict the final prices on df_d_val_scaled
y_val_pred_xgb_final = xgb_final.predict(df_d_val_scaled)

# Create a DataFrame for submission
df_submission_xgb_grid = pd.DataFrame({'Id': df_val['Id'], 'Predicted_Sale_Price_For_Test_Set': y_val_pred_xgb_final})

# Save the submission to an Excel file
file_name_xgb_grid = 'ECON_576_Competition_XGBoost_final.xlsx'
df_submission_xgb_grid.to_excel(file_name_xgb_grid, index=False)

# Display the submission DataFrame
df_submission_xgb_grid

Fitting 5 folds for each of 2 candidates, totalling 10 fits
Best parameters from GridSearchCV for XGBoost: {'gamma': 0, 'learning_rate': 0.1, 'max_depth': 4, 'n_estimators': 200, 'reg_alpha': 180, 'reg_lambda': 140, 'subsample': 0.7}
Best mean cross-validated score from GridSearchCV for XGBoost: 0.8609273707604188
R-squared on local test set using XGBoost with best parameters from GridSearchCV: 0.8762046125606764


,Id,Predicted_Sale_Price_For_Test_Set
0,951,106925.710938
1,952,186613.859375
2,953,297122.406250
3,954,285914.875000
4,955,100032.398438
...,...,...
495,1446,194644.625000
496,1447,221137.906250
497,1448,136423.312500
498,1449,89933.109375


In [ ]:
import xgboost as xgb
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import r2_score
import pandas as pd

# Assuming you have X_scaled, y, X_train_scaled, y_train, X_test_scaled, y_test, df_d_val_scaled, df_val

# Define the parameter grid for XGBoost
param_grid_xgb = {
    'n_estimators': [400, 500, 700],
    'learning_rate': [0.1, 0.05, 0.01],
    'max_depth': [3, 4, 5],
    'subsample': [0.7, 0.8, 0.9],
    'reg_alpha': [90, 110, 170, 200],
    'reg_lambda': [90, 110, 170, 200],
    'colsample_bytree': [0.7, 0.8],
}

# Create an XGBoost regressor
xgb_regressor = xgb.XGBRegressor()

# Create a GridSearchCV object
grid_search_xgb = GridSearchCV(xgb_regressor, param_grid_xgb, cv=5, scoring='r2', verbose=1, n_jobs=-1)
grid_search_xgb.fit(X_scaled, y)

# Get the best parameters and corresponding mean cross-validated score
best_params_xgb = grid_search_xgb.best_params_
best_score_xgb = grid_search_xgb.best_score_
print("Best parameters from GridSearchCV for XGBoost:", best_params_xgb)
print("Best mean cross-validated score from GridSearchCV for XGBoost:", best_score_xgb)

# Fit a separate XGBoost model on X_train_scaled with the best parameters from GridSearchCV
xgb_model = xgb.XGBRegressor(**best_params_xgb)
xgb_model.fit(X_train_scaled, y_train)

# Evaluate the model on the local test set (X_test_scaled)
y_test_pred_xgb_grid = xgb_model.predict(X_test_scaled)
r2_xgb_local_test_grid = r2_score(y_test, y_test_pred_xgb_grid)
print("R-squared on local test set using XGBoost with best parameters from GridSearchCV:", r2_xgb_local_test_grid)

# Fit the XGBoost model on the entire training dataset with the best parameters from GridSearchCV
xgb_final = xgb.XGBRegressor(**best_params_xgb)
xgb_final.fit(X_scaled, y)

# Predict the final prices on df_d_val_scaled
y_val_pred_xgb_final = xgb_final.predict(df_d_val_scaled)

# Create a DataFrame for submission
df_submission_xgb_grid = pd.DataFrame({'Id': df_val['Id'], 'Predicted_Sale_Price_For_Test_Set': y_val_pred_xgb_final})

# Save the submission to an Excel file
file_name_xgb_grid = 'ECON_576_Competition_XGBoost_final_extended.xlsx'
df_submission_xgb_grid.to_excel(file_name_xgb_grid, index=False)

# Display the submission DataFrame
df_submission_xgb_grid


Fitting 5 folds for each of 2592 candidates, totalling 12960 fits
